The goal of this EDA is to understand patterns of play, and to prove / disprove the Wardlow directionals.

We have data regarding the Djokovic vs Nadal match in AO 2019: https://www.youtube.com/watch?v=nHBA_K0RooU

In [2]:
import pandas as pd

df = pd.read_csv('events.csv', index_col=0)

df.head()

,rallyid,frameid,strokeid,hitter,receiver,isserve,serve,type,stroke,hitter_x,hitter_y,receiver_x,receiver_y,time
0,1,70877,1,Djokovic,Nadal,True,first,serve,forehand,6.50,-0.24,1.03,27.44,0.00
1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.05,25.59,6.17,1.11,0.92
2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,1.42,2.33,4.75,26.45,2.92
3,2,71488,1,Djokovic,Nadal,True,first,serve,forehand,4.48,-0.01,7.90,26.83,24.44
4,3,71796,1,Djokovic,Nadal,True,second,serve,forehand,4.48,-0.37,7.88,26.47,36.76


I will rename strokeid because it is actually the current stroke count in the rally

In [5]:
df.rename(columns={'strokeid':'rally_stroke_count'}, inplace=True)

df.head()

,rallyid,frameid,rally_stroke_count,hitter,receiver,isserve,serve,type,stroke,hitter_x,hitter_y,receiver_x,receiver_y,time
0,1,70877,1,Djokovic,Nadal,True,first,serve,forehand,6.50,-0.24,1.03,27.44,0.00
1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.05,25.59,6.17,1.11,0.92
2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,1.42,2.33,4.75,26.45,2.92
3,2,71488,1,Djokovic,Nadal,True,first,serve,forehand,4.48,-0.01,7.90,26.83,24.44
4,3,71796,1,Djokovic,Nadal,True,second,serve,forehand,4.48,-0.37,7.88,26.47,36.76


From an initial exploration we can see that we have the hitter and receiver positions, but it may be useful to have the final ball position also (the initial ball position is the same as hitter).

To have the final ball position we need to get the hitter position of the next stroke in the same rallyid

In [11]:
df['initial_ball_x'] = df['hitter_x']

df['initial_ball_y'] = df['hitter_y']

df['final_ball_x'] = df.groupby('rallyid')['hitter_x'].shift(-1)

df['final_ball_y'] = df.groupby('rallyid')['hitter_y'].shift(-1)

df.head()

,rallyid,frameid,rally_stroke_count,hitter,receiver,isserve,serve,type,stroke,hitter_x,hitter_y,receiver_x,receiver_y,time,final_ball_x,final_ball_y
0,1,70877,1,Djokovic,Nadal,True,first,serve,forehand,6.50,-0.24,1.03,27.44,0.00,0.05,25.59
1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.05,25.59,6.17,1.11,0.92,1.42,2.33
2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,1.42,2.33,4.75,26.45,2.92,NaN,NaN
3,2,71488,1,Djokovic,Nadal,True,first,serve,forehand,4.48,-0.01,7.90,26.83,24.44,NaN,NaN
4,3,71796,1,Djokovic,Nadal,True,second,serve,forehand,4.48,-0.37,7.88,26.47,36.76,7.40,24.79


Great! Seems like we have all the final info about all balls except the final ball of each rally.

To get all information about each event we should merge all other csvs. Let's explore them.

In [52]:
df2 = pd.read_csv('points.csv', index_col=0)

df2.head()

,rallyid,server,returner,winner,reason,serve,strokes,totaltime,x,y,score
0,1,Djokovic,Nadal,Djokovic,winner,first,3,0.92,1.92,21.96,"0:0, 15:0"
2,3,Djokovic,Nadal,Djokovic,out,second,4,4.16,3.33,-0.39,"0:0, 30:0"
4,5,Djokovic,Nadal,Djokovic,ace,second,2,0.40,1.62,17.18,"0:0, 40:0"
5,6,Djokovic,Nadal,Djokovic,net,first,10,10.40,6.41,12.78,"1:0, 0:0"
7,8,Nadal,Djokovic,Djokovic,out,second,7,7.20,0.50,18.67,"1:0, 15:0"


points.csv appears to have every score change, including the reason why the score changed, the score after the rally, and also the final position of the ball. Perfect!

5 notes:
- serve should be removed as we have this info in the main df
- totaltime appears to not include the final hit (if we check the 1st rally the last hit is at 2.92, and totaltime is 0.92). I will remove it and recalculate it.
- reason, strokes, x and y should be renamed for clarity
- split the score into initial score and final score
- score will be split in game_djokovic, game_nadal, points_djokovic and points_nadal (careful because the point scores change according to the server)

Here we drop serve and totaltime, and rename the columns who might be ambiguous.

In [53]:
df2.drop(columns=['serve', 'totaltime'], inplace=True)

df2.rename(columns={
    'reason': 'winner_rally_reason',
    'strokes': 'total_rally_strokes',
    'x': 'final_ball_rally_x',
    'y': 'final_ball_rally_y',
}, inplace=True)

df2.head()

,rallyid,server,returner,winner,winner_rally_reason,total_rally_strokes,final_ball_rally_x,final_ball_rally_y,score
0,1,Djokovic,Nadal,Djokovic,winner,3,1.92,21.96,"0:0, 15:0"
2,3,Djokovic,Nadal,Djokovic,out,4,3.33,-0.39,"0:0, 30:0"
4,5,Djokovic,Nadal,Djokovic,ace,2,1.62,17.18,"0:0, 40:0"
5,6,Djokovic,Nadal,Djokovic,net,10,6.41,12.78,"1:0, 0:0"
7,8,Nadal,Djokovic,Djokovic,out,7,0.50,18.67,"1:0, 15:0"


Now I am populating this df with columns with the score of each player. Let's first check how this df deals with the scores of multiple sets.

In [54]:
df2.tail()

,rallyid,server,returner,winner,winner_rally_reason,total_rally_strokes,final_ball_rally_x,final_ball_rally_y,score
198,199,Nadal,Djokovic,Djokovic,winner,6,2.44,4.71,"6:3 6:2 5:3, 15:15"
200,201,Nadal,Djokovic,Djokovic,net,9,4.26,11.57,"6:3 6:2 5:3, 30:15"
201,202,Nadal,Djokovic,Djokovic,winner,4,1.88,3.36,"6:3 6:2 5:3, 40:15"
203,204,Nadal,Djokovic,Nadal,out,12,8.45,-0.56,"6:3 6:2 5:3, 40:30"
205,206,Nadal,Djokovic,Djokovic,out,5,0.59,23.86,"6:3 6:2 6:3, 0:0"


Seems like a space is added everytime a new set is created. So, we need to check for this when calculating the current game of the set and also how many sets each have won.

In [94]:
import numpy as np


df2["games_djokovic_after_rally"] = df2["score"].str.split(", ").str.get(0).str.split(" ").str.get(-1).str.split(":").str.get(0)

df2["games_nadal_after_rally"] = df2["score"].str.split(", ").str.get(0).str.split(" ").str.get(-1).str.split(":").str.get(1)

df2["points_djokovic_after_rally"] = np.where(
    df2["server"].str.contains("Djokovic"),
    df2["score"].str.split(", ").str.get(1).str.split(":").str.get(0),
    df2["score"].str.split(", ").str.get(1).str.split(":").str.get(1),
)

df2["points_nadal_after_rally"] = np.where(
    df2["server"].str.contains("Nadal"),
    df2["score"].str.split(", ").str.get(1).str.split(":").str.get(0),
    df2["score"].str.split(", ").str.get(1).str.split(":").str.get(1),
)

df2.fillna(0, inplace=True)

df2["games_djokovic_after_rally"] = df2["games_djokovic_after_rally"].astype(int)
df2["games_nadal_after_rally"] = df2["games_nadal_after_rally"].astype(int)

df2.tail()

,rallyid,server,returner,winner,winner_rally_reason,total_rally_strokes,final_ball_rally_x,final_ball_rally_y,score,games_djokovic_after_rally,games_nadal_after_rally,points_djokovic_after_rally,points_nadal_after_rally,sets_djokovic_after_rally
198,199,Nadal,Djokovic,Djokovic,winner,6,2.44,4.71,"6:3 6:2 5:3, 15:15",5,3,15,15,1.0
200,201,Nadal,Djokovic,Djokovic,net,9,4.26,11.57,"6:3 6:2 5:3, 30:15",5,3,15,30,1.0
201,202,Nadal,Djokovic,Djokovic,winner,4,1.88,3.36,"6:3 6:2 5:3, 40:15",5,3,15,40,1.0
203,204,Nadal,Djokovic,Nadal,out,12,8.45,-0.56,"6:3 6:2 5:3, 40:30",5,3,30,40,1.0
205,206,Nadal,Djokovic,Djokovic,out,5,0.59,23.86,"6:3 6:2 6:3, 0:0",6,3,0,0,1.0


In [93]:
condition = (df2['games_djokovic_after_rally'] == 5)
df2['sets_djokovic_after_rally'] = 0
df2['sets_djokovic_after_rally'] = np.where(condition, df2['sets_djokovic_after_rally'] + 1, df2['sets_djokovic_after_rally'].shift())

df2.iloc[42:48]

,rallyid,server,returner,winner,winner_rally_reason,total_rally_strokes,final_ball_rally_x,final_ball_rally_y,score,games_djokovic_after_rally,games_nadal_after_rally,points_djokovic_after_rally,points_nadal_after_rally,sets_djokovic_after_rally
62,63,Djokovic,Nadal,Djokovic,net,22,3.00,12.51,"5:3, 40:15",5,3,40,15,1.0
63,64,Djokovic,Nadal,Djokovic,net,4,2.81,12.03,"6:3 0:0, 0:0",5,3,0,0,1.0
64,65,Nadal,Djokovic,Djokovic,out,5,-1.04,20.45,"6:3 0:0, 15:0",0,0,0,15,0.0
65,66,Nadal,Djokovic,Nadal,winner,3,1.67,20.06,"6:3 0:0, 15:15",0,0,15,15,0.0
66,67,Nadal,Djokovic,Nadal,out,2,8.51,-1.35,"6:3 0:0, 15:30",0,0,30,15,0.0
69,70,Nadal,Djokovic,Djokovic,winner,4,1.40,1.59,"6:3 0:0, 30:30",0,0,30,30,0.0


In [49]:
df2["games_djokovic_before_rally"] = df2["games_djokovic_after_rally"].shift(1)
df2["games_nadal_before_rally"] = df2["games_nadal_after_rally"].shift(1)
df2["points_djokovic_before_rally"] = df2["points_djokovic_after_rally"].shift(1)
df2["points_nadal_before_rally"] = df2["points_nadal_after_rally"].shift(1)

df2.fillna(0, inplace=True)

df2.drop(columns="score", inplace=True)

df2.head()

,rallyid,server,returner,winner,winner_rally_reason,total_rally_strokes,final_ball_rally_x,final_ball_rally_y,games_djokovic,games_nadal,points_djokovic,points_nadal,games_djokovic_after_rally,games_nadal_after_rally,points_djokovic_after_rally,points_nadal_after_rally,games_djokovic_before_rally,games_nadal_before_rally,points_djokovic_before_rally,points_nadal_before_rally
0,1,Djokovic,Nadal,Djokovic,winner,3,1.92,21.96,0,0,15,0,0,0,15,0,0,0,0,0
2,3,Djokovic,Nadal,Djokovic,out,4,3.33,-0.39,0,0,30,0,0,0,30,0,0,0,15,0
4,5,Djokovic,Nadal,Djokovic,ace,2,1.62,17.18,0,0,40,0,0,0,40,0,0,0,30,0
5,6,Djokovic,Nadal,Djokovic,net,10,6.41,12.78,1,0,0,0,1,0,0,0,0,0,40,0
7,8,Nadal,Djokovic,Djokovic,out,7,0.50,18.67,1,0,0,15,1,0,0,15,1,0,0,0
